In [ ]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

from Code.ResidualAttentionNetwork import ResidualAttentionNetwork
import numpy as np
import pandas as pd 
import os
import glob
from datetime import datetime
from pathlib import Path

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers 

import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

# Organize Data 

In [ ]:
# Construct the files to be placed inside a dataframe
def construct_data(filepath):
    data = {}
    data['filepath'] = filepath
    data['class'] = str(Path(filepath).parent).split('/')[-1]
    return data

In [ ]:
data_dir = "/pylon5/cc5614p/deopha32/eye_images/"

data_files=[filename for filename in glob.iglob(data_dir + '*/*/*', recursive=True)]

# Place inside dataframe
data = [construct_data(file) for file in data_files]
eye_df = pd.DataFrame(data)

In [ ]:
X = eye_df['filepath']
y = eye_df['class']

# 60-20-20 Split: https://stackoverflow.com/a/38251213/9221241
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train)

model_train_data = pd.concat([X_train, y_train], axis=1)
model_train_data = pd.DataFrame(model_train_data)

model_val_data = pd.concat([X_val, y_val], axis=1)
model_val_data = pd.DataFrame(model_val_data)

model_test_data = pd.concat([X_test, y_test], axis=1)
model_test_data = pd.DataFrame(model_test_data)

In [ ]:
class_names = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

weights = class_weight.compute_class_weight('balanced',
                                             class_names,
                                             y_train)

class_weights = {class_names[index]:weights[index] for index in range(len(class_names))}

In [ ]:
# Save Dataframes for future use
model_train_data.to_csv("/home/deopha32/EyeDiseaseClassification/Data/training_data", index=False)
model_val_data.to_csv("/home/deopha32/EyeDiseaseClassification/Data/validatation_data", index=False)
model_test_data.to_csv("/home/deopha32/EyeDiseaseClassification/Data/testing_data", index=False)

# Network Metadata

In [ ]:
IMAGE_WIDTH=32
IMAGE_HEIGHT=32
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1
IMAGE_SHAPE=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)

batch_size=32

num_classes = 4

epochs = 500

# Image Generators

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255.)

train_generator=train_datagen.flow_from_dataframe(
    dataframe=model_train_data,
    x_col="filepath",
    y_col="class",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    validate_filenames=False
)

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255.)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=model_val_data,
    x_col="filepath",
    y_col="class",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    validate_filenames=False
)

In [ ]:
class_indices = train_generator.class_indices
new_class_weights = dict((class_indices[key], value) for (key, value) in class_weights.items())

# Train Model

In [ ]:
curr_time = f'{datetime.now():%H-%M-%S%z_%m%d%Y}'

model_path = "/pylon5/cc5614p/deopha32/Saved_Models/eye-model_{}.h5".format(curr_time)
logger_path = "/pylon5/cc5614p/deopha32/Saved_Models/eye-model-history_{}.csv".format(curr_time)

In [ ]:
checkpoint = ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True)
csv_logger = CSVLogger(logger_path, append=True)

callbacks = [checkpoint, csv_logger]

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
# Model Training
with tf.device('/gpu:0'):
    model = ResidualAttentionNetwork(
                input_shape=IMAGE_SHAPE, 
                n_classes=num_classes, 
                activation='softmax').build_model()
    
    model.compile(optimizer=optimizers.RMSprop(lr=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit_generator(generator=train_generator, class_weight=new_class_weights,
                    steps_per_epoch=STEP_SIZE_TRAIN, verbose=1, callbacks=callbacks,
                    validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                    epochs=epochs, use_multiprocessing=True, workers=40)